<a href="https://colab.research.google.com/github/gabrielbauer/T1_ML/blob/main/Trabalho_Aprendizado_de_M%C3%A1quina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Componentes do grupo:
- Gabriel Bauer de Oliveira - 00275614
- Vitor Hugo da Silveira Fuerstenau Maciel - 00275605
- Fábio Petkowicz - 

Base de dados: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset?resource=download&select=True.csv

##Inspiration
Can you use this data set to make an algorithm able to determine if an article is fake news or not?

In [34]:
#Imports
import pandas as pd
from IPython.display import display
from typing import List
from typing import Tuple
import random as random

In [ ]:
#@title Gather dataset.zip
file_id = "1LBu6G9C4E5qODJpFuLeRdi6VKrIHaLLJ" 
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html


from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1gLBqEWEBQDYbKCDigHnUXNTkzl-OslSO

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

fileId = drive.CreateFile({'id': file_id }) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])  
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file

!unzip {fileId['title']} -d "sample_data/"

In [26]:
true_news_dataframe = pd.read_csv("sample_data/True.csv")
fake_news_dataframe = pd.read_csv("sample_data/Fake.csv")

true_news_dataframe['true'] = 1
fake_news_dataframe['true'] = 0

news_dataframe = pd.concat([true_news_dataframe, fake_news_dataframe])
news_dataframe.head()

,title,text,subject,date,true
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


Verificação de colunas com campos vazios

In [28]:
colunas = ['title', 'text', 'subject', 'date', 'true']
dataframes = [news_dataframe]

valido = 1
for dataframe in dataframes:
  for coluna in colunas:
   if (dataframe.loc[dataframe[coluna] == None].empty == False):
      valido = -1
      break

print("Dataframes sem colunas vazias.") if valido == 1 else print("Dataframe com colunas vazias.")


Dataframes sem colunas vazias.


Realizar uma implementação própria da estratégia de k-fold cross validation, que seja genérica o suficiente para permitir o uso de diferentes valores de k (número de folds). A estratégia deve ser aplicada de forma estratificada, isto é, mantendo-se a proporção original de exemplos por classe em todos os folds, independente do número de classes do problema. É aconselhável que os grupos estudem as funções prontas a serem utilizadas para treinamento dos modelos, a fim de implementarem sua função para validação cruzada de forma compatível com as mesmas (principalmente em relação a estruturas de dados).

In [ ]:
#KFOLDS
"""
@parameters
k: numero de folds
dataframe: dataframe com os dados
@return
[pd.DataFrame]: lista dos folds de noticias verdadeiras e falsas
"""
def create_kfolds(k = int, dataframe = pd.DataFrame) -> List[List[pd.DataFrame]]:
  true_news_dataframe = dataframe.loc[dataframe['true'] == 1]
  fake_news_dataframe = dataframe.loc[dataframe['true'] == 0]
  
  true_news_size = true_news_dataframe.shape[0]
  fake_news_size = fake_news_dataframe.shape[0]

  random.seed()

#generating true news folds
  true_news_folds = []

  first_index_true = random.randrange(0, true_news_size)
  fold_size = true_news_size//k

  current_index = first_index_true
  for _ in range (0, k):
    if(current_index + fold_size >= true_news_size):
      offset = true_news_size - current_index

      #quantas instancias do fold se encontram até o final do dataframe
      #devem ser alocadas fold_size - offset instancias no inicio do dataframe
      
      fold_start_index = current_index
      fold_end_index = true_news_size
      fold_end = pd.DataFrame(true_news_dataframe[fold_start_index:fold_end_index])

      fold_start_index = 0
      fold_end_index = fold_size - offset
      fold_start = pd.DataFrame(true_news_dataframe[fold_start_index:fold_end_index])
      
      fold = pd.concat([fold_end, fold_start])

      true_news_folds.append(fold)

      current_index = fold_size - offset

    else:
      fold_start_index = current_index
      fold_end_index = current_index + fold_size
      fold = pd.DataFrame(true_news_dataframe[fold_start_index:fold_end_index])

      true_news_folds.append(fold)

      current_index = current_index + fold_size

#generating fake news folds
  fake_news_folds = []

  first_index_fake = random.randrange(0, fake_news_size)
  fold_size = fake_news_size//k

  current_index = first_index_fake
  for _ in range (0, k):
    if(current_index + fold_size >= fake_news_size):
      offset = fake_news_size - current_index

      #quantas instancias do fold se encontram até o final do dataframe
      #devem ser alocadas fold_size - offset instancias no inicio do dataframe
      
      fold_start_index = current_index
      fold_end_index = fake_news_size
      fold_end = pd.DataFrame(fake_news_dataframe[fold_start_index:fold_end_index])

      fold_start_index = 0
      fold_end_index = fold_size - offset
      fold_start = pd.DataFrame(fake_news_dataframe[fold_start_index:fold_end_index])
      
      fold = pd.concat([fold_end, fold_start])

      fake_news_folds.append(fold)

      current_index = fold_size - offset

    else:
      fold_start_index = current_index
      fold_end_index = current_index + fold_size
      fold = pd.DataFrame(fake_news_dataframe[fold_start_index:fold_end_index])

      fake_news_folds.append(fold)
      current_index = current_index + fold_size

  return [pd.concat([true_df, fake_df]) for true_df, fake_df in zip(true_news_folds, fake_news_folds)]

fold_list = create_kfolds(10, news_dataframe)
for fold in fold_list:
  display(fold)